In [1]:
import os
from google.colab import drive
import subprocess
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#!export CUDA_PATH=/usr/local/cuda-9.0
!pip install -q Ninja
proj_path = '/content/drive/MyDrive/lewagon/project'
pkg_path = os.path.join(proj_path, 'music-translation')
os.chdir(pkg_path)
subprocess.run(f'pip install -r requirements.txt', shell = True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 2.8 MB/s eta 0:00:00


CompletedProcess(args='pip install -r requirements.txt', returncode=0)

In [3]:
c_path = os.path.join(pkg_path, 'src/nv-wavenet')
os.chdir(c_path)
! rm -rf .build_dir build dist nv_wavenet_ext.egg-info ../.build_dir
subprocess.run(f'python setup.py install', shell = True)

CompletedProcess(args='python setup.py install', returncode=0)

In [4]:
from pathlib import Path
import librosa
import torch
from argparse import ArgumentParser
import matplotlib
import h5py
import tqdm
from IPython.display import Audio, display

import sys
#sys.path += ['../src']
note_path = os.path.join(pkg_path, 'notebooks')
sys.path.append(note_path+'/../src')

import myutils
import wavenet_models
from myutils import save_audio
from wavenet import WaveNet
from wavenet_generator import WavenetGenerator
from nv_wavenet_generator import NVWavenetGenerator

In [5]:
src_path = os.path.join(pkg_path, 'src')
os.chdir(src_path)
!pwd

/content/drive/MyDrive/lewagon/project/music-translation/src


In [6]:
test_path = os.path.join(proj_path, 'data/test_data/2556.wav')
print(test_path)

/content/drive/MyDrive/lewagon/project/data/test_data/2556.wav


In [7]:
# Inference params
che_path = os.path.join(proj_path, 'music-translation_pretrain')
checkpoint = Path(f'{che_path}/lastmodel')
decoders = [0, 1, 2, 3, 4, 5]
batch_size = 1
rate = 16000
split_size = 20
file_paths = [Path(f'{note_path}/test.wav')]
#file_paths = [Path(f'{test_path}')]

In [8]:
def disp(x, decoder_ix):
    wav = myutils.inv_mu_law(x.cpu().numpy())
    print(f'Decoder: {decoder_ix}')
    print(f'X min: {x.min()}, max: {x.max()}')

    display(Audio(wav.squeeze(), rate=rate))

def extract_id(path):
    decoder_id = str(path)[:-4].split('_')[-1]
    return int(decoder_id)

In [9]:
print('Starting')
matplotlib.use('agg')

checkpoints = checkpoint.parent.glob(checkpoint.name + '_*.pth')
checkpoints = [c for c in checkpoints if extract_id(c) in decoders]
assert len(checkpoints) >= 1, "No checkpoints found."

model_args = torch.load(checkpoint.parent / 'args.pth')[0]
encoder = wavenet_models.Encoder(model_args)
encoder.load_state_dict(torch.load(checkpoints[0])['encoder_state'])
encoder.eval()
encoder = encoder.cuda()

decoders = []
decoder_ids = []
for checkpoint in checkpoints:
    decoder = WaveNet(model_args)
    decoder.load_state_dict(torch.load(checkpoint)['decoder_state'])
    decoder.eval()
    decoder = decoder.cuda()
    decoder = NVWavenetGenerator(decoder, rate * (split_size // 20), batch_size, 3)

    decoders += [decoder]
    decoder_ids += [extract_id(checkpoint)]

Starting
Failed loading nv_wavenet_ext, building dynamically


RuntimeError: Error building extension 'wavenet_cu_79y8r5ux': [1/4] c++ -MMD -MF matrix.o.d -DTORCH_EXTENSION_NAME=wavenet_cu_79y8r5ux -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /usr/local/lib/python3.10/dist-packages/torch/include -isystem /usr/local/lib/python3.10/dist-packages/torch/include/torch/csrc/api/include -isystem /usr/local/lib/python3.10/dist-packages/torch/include/TH -isystem /usr/local/lib/python3.10/dist-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /usr/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /content/drive/MyDrive/lewagon/project/music-translation/src/nv-wavenet/matrix.cpp -o matrix.o 
[2/4] /usr/local/cuda/bin/nvcc  -DTORCH_EXTENSION_NAME=wavenet_cu_79y8r5ux -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /usr/local/lib/pyt

In [ ]:
xs = []

for file_path in file_paths:
    data, rate = librosa.load(file_path, sr=16000)
    print(data.shape)
    assert rate == 16000
    data = myutils.mu_law(data)
    xs.append(torch.tensor(data).unsqueeze(0).float().cuda())

xs = torch.stack(xs).contiguous()
print(f'xs size: {xs.size()}')

(2423328,)
xs size: torch.Size([1, 1, 80000])


In [ ]:
yy = {}
with torch.no_grad():
    zz = []
    for xs_batch in torch.split(xs, batch_size):
        zz += [encoder(xs_batch)]
    zz = torch.cat(zz, dim=0)

    with myutils.timeit("Generation timer"):
        for i, decoder_id in enumerate(decoder_ids):
            yy[decoder_id] = []
            decoder = decoders[i]
            for zz_batch in torch.split(zz, batch_size):
                print(zz_batch.shape)
                splits = torch.split(zz_batch, split_size, -1)
                audio_data = []
                decoder.reset()
                for cond in tqdm.tqdm_notebook(splits):
                    audio_data += [decoder.generate(cond).cpu()]
                audio_data = torch.cat(audio_data, -1)
                yy[decoder_id] += [audio_data]
            yy[decoder_id] = torch.cat(yy[decoder_id], dim=0)

torch.Size([1, 64, 100])


<ipython-input-51-d7c226caaa4f>:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for cond in tqdm.tqdm_notebook(splits):


  0%|          | 0/5 [00:00<?, ?it/s]

torch.Size([1, 64, 100])


  0%|          | 0/5 [00:00<?, ?it/s]

...

torch.Size([1, 64, 100])


  0%|          | 0/5 [00:00<?, ?it/s]

torch.Size([1, 64, 100])


  0%|          | 0/5 [00:00<?, ?it/s]

Generation timer took 29286.905765533447 ms


In [ ]:
display(Audio(data, rate=rate))

In [ ]:
for decoder_ix, decoder_result in yy.items():
    for sample_result, filepath in zip(decoder_result, file_paths):
        disp(sample_result, decoder_ix)

Decoder: 0
X min: 21, max: 229


Decoder: 1
X min: 20, max: 232


Decoder: 4
X min: 14, max: 240


Decoder: 3
X min: 32, max: 225


Decoder: 2
X min: 45, max: 209


Decoder: 5
X min: 12, max: 244
